In [1]:
! pip install polars
! pip install numpy

  Using cached polars-1.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
Using cached polars-1.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.0 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [3]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [4]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: None


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [5]:
data = data.with_columns(
    pl.col("timestamp")
    .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f", strict=False)
    .alias("timestamp")
)

/tmp/ipykernel_1284/421030682.py:3: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  .str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f", strict=False)


In [6]:
data.collect()

car_id,timestamp,latitude,longitude,notification
str,datetime[μs],f64,f64,str
"""car_0""",null,23.733188,-94.23376,"""tire_pressure"""
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,null
"""car_0""",2023-01-01 00:00:05.000282,23.733188,-94.23376,"""low_fuel"""
"""car_0""",2023-01-01 00:11:23.000597,23.733188,-94.23376,"""low_fuel"""
"""car_0""",null,23.727644,-94.233698,"""engine_check"""
…,…,…,…,…
"""car_999""",2023-01-08 00:01:36.000407,24.033558,-114.110391,"""engine_check"""
"""car_999""",2023-01-08 00:01:43.000543,24.033558,-114.110391,"""low_fuel"""
"""car_999""",2023-01-08 00:01:51.000490,24.033558,-114.110391,"""low_fuel"""


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [7]:
data = data.with_columns(
    pl.col("timestamp").diff().over("car_id").alias("notification_time")
 )

In [8]:
data.collect()

car_id,timestamp,latitude,longitude,notification,notification_time
str,datetime[μs],f64,f64,str,duration[μs]
"""car_0""",null,23.733188,-94.23376,"""tire_pressure""",null
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,null,null
"""car_0""",2023-01-01 00:00:05.000282,23.733188,-94.23376,"""low_fuel""",-27m -40s -6µs
"""car_0""",2023-01-01 00:11:23.000597,23.733188,-94.23376,"""low_fuel""",11m 18s 315µs
"""car_0""",null,23.727644,-94.233698,"""engine_check""",null
…,…,…,…,…,…
"""car_999""",2023-01-08 00:01:36.000407,24.033558,-114.110391,"""engine_check""",4s 999415µs
"""car_999""",2023-01-08 00:01:43.000543,24.033558,-114.110391,"""low_fuel""",7s 136µs
"""car_999""",2023-01-08 00:01:51.000490,24.033558,-114.110391,"""low_fuel""",7s 999947µs


2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [9]:
# Radius of Earth in kilometers
R = 6371.0

# Convert latitude and longitude columns to radians
data = data.with_columns([
    (pl.col("latitude") * np.pi / 180).alias("lat_rad"),
    (pl.col("longitude") * np.pi / 180).alias("lon_rad"),
])

# Calculate Haversine distance with correct usage of sin, cos, and arcsin as chained methods
data = data.with_columns([
    (2 * R * (
        (pl.col("lat_rad").shift(-1) - pl.col("lat_rad")).sin().pow(2) +
        (pl.col("lat_rad").cos() * pl.col("lat_rad").shift(-1).cos() *
         (pl.col("lon_rad").shift(-1) - pl.col("lon_rad")).sin().pow(2))
    ).sqrt().arcsin()).over("car_id").alias("distance")
])

# Drop the intermediate columns if no longer needed
data = data.drop(["lat_rad", "lon_rad"])

# Collect and display the results
data_df = data.collect()
data_df

car_id,timestamp,latitude,longitude,notification,notification_time,distance
str,datetime[μs],f64,f64,str,duration[μs],f64
"""car_0""",null,23.733188,-94.23376,"""tire_pressure""",null,0.0
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,null,null,0.0
"""car_0""",2023-01-01 00:00:05.000282,23.733188,-94.23376,"""low_fuel""",-27m -40s -6µs,0.0
"""car_0""",2023-01-01 00:11:23.000597,23.733188,-94.23376,"""low_fuel""",11m 18s 315µs,1.233058
"""car_0""",null,23.727644,-94.233698,"""engine_check""",null,0.0
…,…,…,…,…,…,…
"""car_999""",2023-01-08 00:01:36.000407,24.033558,-114.110391,"""engine_check""",4s 999415µs,0.0
"""car_999""",2023-01-08 00:01:43.000543,24.033558,-114.110391,"""low_fuel""",7s 136µs,0.0
"""car_999""",2023-01-08 00:01:51.000490,24.033558,-114.110391,"""low_fuel""",7s 999947µs,0.0


3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [10]:
# Step 1: Fill nulls, ensuring `notification_time` stays as `duration[μs]`
data = data.with_columns([
    pl.col("timestamp").fill_null(strategy="forward").cast(pl.Datetime)
])

# Check if there are any remaining null values in 'timestamp'
null_timestamps = data.filter(pl.col("timestamp").is_null()).collect()
print("Rows with null timestamps:", null_timestamps.shape[0])

Rows with null timestamps: 1


In [11]:
data = data.with_columns([
    pl.col("timestamp").fill_null(strategy="backward").cast(pl.Datetime)
])

In [12]:
null_timestamps = data.filter(pl.col("timestamp").is_null()).collect()
print("Rows with null timestamps:", null_timestamps.shape[0])
null_timestamps

Rows with null timestamps: 0


car_id,timestamp,latitude,longitude,notification,notification_time,distance
str,datetime[μs],f64,f64,str,duration[μs],f64


In [13]:
data = data.with_columns([
    pl.col("notification_time").fill_null(pl.duration(microseconds=0)),
    pl.col("distance").fill_null(0.0)
])
data.collect()

car_id,timestamp,latitude,longitude,notification,notification_time,distance
str,datetime[μs],f64,f64,str,duration[μs],f64
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,"""tire_pressure""",0µs,0.0
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,null,0µs,0.0
"""car_0""",2023-01-01 00:00:05.000282,23.733188,-94.23376,"""low_fuel""",-27m -40s -6µs,0.0
"""car_0""",2023-01-01 00:11:23.000597,23.733188,-94.23376,"""low_fuel""",11m 18s 315µs,1.233058
"""car_0""",2023-01-01 00:11:23.000597,23.727644,-94.233698,"""engine_check""",0µs,0.0
…,…,…,…,…,…,…
"""car_999""",2023-01-08 00:01:36.000407,24.033558,-114.110391,"""engine_check""",4s 999415µs,0.0
"""car_999""",2023-01-08 00:01:43.000543,24.033558,-114.110391,"""low_fuel""",7s 136µs,0.0
"""car_999""",2023-01-08 00:01:51.000490,24.033558,-114.110391,"""low_fuel""",7s 999947µs,0.0


In [14]:
data.sort(["car_id", "timestamp"])
data.collect()

car_id,timestamp,latitude,longitude,notification,notification_time,distance
str,datetime[μs],f64,f64,str,duration[μs],f64
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,"""tire_pressure""",0µs,0.0
"""car_0""",2023-01-01 00:27:45.000288,23.733188,-94.23376,null,0µs,0.0
"""car_0""",2023-01-01 00:00:05.000282,23.733188,-94.23376,"""low_fuel""",-27m -40s -6µs,0.0
"""car_0""",2023-01-01 00:11:23.000597,23.733188,-94.23376,"""low_fuel""",11m 18s 315µs,1.233058
"""car_0""",2023-01-01 00:11:23.000597,23.727644,-94.233698,"""engine_check""",0µs,0.0
…,…,…,…,…,…,…
"""car_999""",2023-01-08 00:01:36.000407,24.033558,-114.110391,"""engine_check""",4s 999415µs,0.0
"""car_999""",2023-01-08 00:01:43.000543,24.033558,-114.110391,"""low_fuel""",7s 136µs,0.0
"""car_999""",2023-01-08 00:01:51.000490,24.033558,-114.110391,"""low_fuel""",7s 999947µs,0.0


In [15]:
data = data.with_columns([
    pl.col("timestamp").fill_null(strategy="forward").cast(pl.Datetime),
    pl.col("notification_time").fill_null(pl.duration(microseconds=0)),
    pl.col("distance").fill_null(0.0).cast(pl.Float64)
]).sort(["car_id", "timestamp"]).collect() 

In [16]:
data = data.lazy()

In [17]:
x_minutes = 5
data_aggregated = (data
    .group_by_dynamic(
        index_column="timestamp",
        every=f"{x_minutes}m",
        by="car_id"
    )
    .agg([
        pl.col("notification_time").mean().alias("mean_notification_time"),
        pl.col("notification_time").median().alias("median_notification_time"),
        pl.col("notification_time").var().alias("var_notification_time"),
        pl.col("notification_time").max().alias("max_notification_time"),
        pl.col("notification_time").min().alias("min_notification_time"),
        pl.col("distance").mean().alias("mean_distance"),
        pl.col("distance").median().alias("median_distance"),
        pl.col("distance").var().alias("var_distance"),
        pl.col("distance").max().alias("max_distance"),
        pl.col("distance").min().alias("min_distance"),
    ])
)

# Collect and display results
result = data_aggregated.collect()
result

/tmp/ipykernel_1284/2345657152.py:3: DeprecationWarning: The argument `by` for `LazyFrame.group_by_dynamic` is deprecated. It has been renamed to `group_by`.
  .group_by_dynamic(


car_id,timestamp,mean_notification_time,median_notification_time,var_notification_time,max_notification_time,min_notification_time,mean_distance,median_distance,var_distance,max_distance,min_distance
str,datetime[μs],duration[μs],duration[μs],duration[μs],duration[μs],duration[μs],f64,f64,f64,f64,f64
"""car_0""",2023-01-01 00:00:00,-27m -40s -6µs,-27m -40s -6µs,null,-27m -40s -6µs,-27m -40s -6µs,0.0,0.0,null,0.0,0.0
"""car_0""",2023-01-01 00:10:00,5m 39s 157µs,5m 39s 157µs,2660210d 19h 19m 30s 49600µs,11m 18s 315µs,0µs,0.616529,0.616529,0.760217,1.233058,0.0
"""car_0""",2023-01-01 00:25:00,0µs,0µs,0µs,0µs,0µs,0.0,0.0,0.0,0.0,0.0
"""car_0""",2023-01-01 00:30:00,-4m -22s -166690µs,-47s -238µs,2504122d 2h 42m 20s 9248µs,0µs,-19m -42s -58µs,0.438505,0.0,0.482176,1.542937,0.0
"""car_0""",2023-01-01 00:35:00,-8m -59s -500435µs,-8m -59s -500435µs,866720d 14h 10m 57s 84464µs,-5m -46s -230µs,-12m -13s -641µs,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…
"""car_999""",2023-01-07 23:40:00,5s 545442µs,5s 423µs,76d 15h 38m 284834µs,9s 999702µs,1s 11µs,0.0,0.0,0.0,0.0,0.0
"""car_999""",2023-01-07 23:45:00,14s 629643µs,6s 304µs,51865d 2h 11m 27s 560657µs,8m 17s 75µs,1s 36µs,0.021358,0.0,0.024634,1.153354,0.0
"""car_999""",2023-01-07 23:50:00,5s 716977µs,5s 596µs,98d 23h 48m 14s 171256µs,9s 999835µs,0µs,0.0,0.0,0.0,0.0,0.0
